In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Sat Mar  5 17:12:29 2022

@author: RRZ
"""
# -*- coding: utf-8 -*-
"""
Created on Mon Feb 21 18:09:48 2022

@author: RRZ
"""
from joblib import dump,load
import math
import pandas as pd
import numpy as np
import random
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

df_raw_train = pd.read_csv('../data/raw/train.csv')
df_raw_test = pd.read_csv('../data/raw/test.csv')

train_df_size = 0.8
rand_state_ind = 42
validation_df_size = 0.2
scoring = 'roc_auc'
cv = 10

df_cleaned_train = df_raw_train.copy()

target = df_cleaned_train.pop('TARGET_5Yrs')
IDlist_train = df_cleaned_train.pop('Id')
df_col_names = df_cleaned_train.columns
train_dataset_size = IDlist_train.size

scaler = StandardScaler()
array_cleaned_train = scaler.fit_transform(df_cleaned_train)
df_cleaned2_train = pd.DataFrame(array_cleaned_train,columns=df_col_names)

df_cleaned_test = df_raw_test.copy()
IDlist_test = df_cleaned_test.pop('Id')

array_cleaned_test = scaler.transform(df_cleaned_test) 
df_cleaned2_test = pd.DataFrame(array_cleaned_test,columns=df_col_names)

y_train_pos_count = sum(target)
y_train_neg_count = target.size - y_train_pos_count


#Upsampling
from sklearn.utils import resample
df_train_posclass = df_raw_train[df_raw_train['TARGET_5Yrs']==1]
df_train_negclass = df_raw_train[df_raw_train['TARGET_5Yrs']==0]
df_train_negclass_upsampled = resample(df_train_negclass, replace=True, n_samples=y_train_pos_count, random_state=42)

df_cleaned3_train = pd.concat([df_train_posclass,df_train_negclass_upsampled])

target_upsampled = df_cleaned3_train.pop('TARGET_5Yrs')
IDlist_train_upsampled = df_cleaned3_train.pop('Id')

df_cleaned3_test = df_cleaned2_test.copy()

#Variable selection
selected_features2 = ['GP','MIN','PTS','FGA','FGM','FG%','REB','FTA','OREB']

#original set
df_cleaned4_train = df_cleaned2_train.copy()
df_cleaned4_train = df_cleaned4_train.loc[:,selected_features2]
df_cleaned4_test = df_cleaned2_train.copy()
df_cleaned4_test = df_cleaned4_test.loc[:,selected_features2]

#Upsampled set
df_cleaned5_train = df_cleaned3_train.copy()
df_cleaned5_train = df_cleaned5_train.loc[:,selected_features2]
df_cleaned5_test = df_cleaned3_train.copy()
df_cleaned5_test = df_cleaned5_test.loc[:,selected_features2]

#Variable selection2
selected_features1 = ['GP','FGM','3P Made','OREB','DREB']

#original set
df_cleaned6_train = df_cleaned2_train.copy()
df_cleaned6_train = df_cleaned6_train.loc[:,selected_features1]
df_cleaned6_test = df_cleaned2_train.copy()
df_cleaned6_test = df_cleaned6_test.loc[:,selected_features1]

#Upsampled set
df_cleaned7_train = df_cleaned3_train.copy()
df_cleaned7_train = df_cleaned7_train.loc[:,selected_features1]
df_cleaned7_test = df_cleaned3_train.copy()
df_cleaned7_test = df_cleaned7_test.loc[:,selected_features1]

def BuildCSVforSubmission(IDlist,PredictionList):
    iErrCode = 0
    fFileName = 'submission.csv'
    if (IDlist.size != 3799) or (PredictionList.size != 3799):
        iErrCode = 1
    if iErrCode == 0:
        if not(isinstance(IDlist, pd.Series)):
            list1 = pd.Series(IDlist)
        else:
            list1 = IDlist.copy()
        if not(isinstance(PredictionList, pd.Series)):
            list2 = pd.Series(PredictionList)
        else:
            list2 = IDlist.copy()
        #list2.round(2)
        #df_out = pd.DataFrame([list1,list2])
        df_out = pd.DataFrame({'Id':list1,'TARGET_5Yrs':list2})
        df_out.to_csv(path_or_buf = '../data/processed/'+fFileName,index=False)
    return iErrCode

from sklearn.metrics import roc_auc_score

from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import GridSearchCV

logRegModel = LogisticRegression(l1_ratio=1.0,penalty = 'elasticnet',max_iter=10000,random_state=42,class_weight='balanced',solver='saga')
cRegStr = np.linspace(start=0,stop=1,num=11)
param_grid11 = {'C':cRegStr}

clf51 = GridSearchCV(estimator=logRegModel,param_grid=param_grid11,cv=cv,scoring=scoring,return_train_score=True,verbose=3)
clf51.fit(df_cleaned2_train, target)
clf51.best_estimator_
clf51.score(df_cleaned2_train, target)
clf51.cv_results_.keys()

dump(clf51,  '../models/logreg_10cv_gridsearch_Cvaluesonly_vardropcompare1.joblib')

clf52 = GridSearchCV(estimator=logRegModel,param_grid=param_grid11,cv=cv,scoring=scoring,return_train_score=True,verbose=3)
clf52.fit(df_cleaned3_train, target_upsampled)
clf52.best_estimator_
clf52.score(df_cleaned3_train, target_upsampled)
clf52.cv_results_.keys()

dump(clf52,  '../models/logreg_10cv_gridsearch_Cvaluesonly_vardropcompare2.joblib')

clf53 = GridSearchCV(estimator=logRegModel,param_grid=param_grid11,cv=cv,scoring=scoring,return_train_score=True,verbose=3)
clf53.fit(df_cleaned4_train, target)
clf53.best_estimator_
clf53.score(df_cleaned4_train, target)
clf53.cv_results_.keys()

dump(clf53,  '../models/logreg_10cv_gridsearch_Cvaluesonly_vardropcompare3.joblib')

clf54 = GridSearchCV(estimator=logRegModel,param_grid=param_grid11,cv=cv,scoring=scoring,return_train_score=True,verbose=3)
clf54.fit(df_cleaned5_train, target_upsampled)
clf54.best_estimator_
clf54.score(df_cleaned5_train, target_upsampled)
clf54.cv_results_.keys()

dump(clf54,  '../models/logreg_10cv_gridsearch_Cvaluesonly_vardropcompare4.joblib')

import xgboost as xgb
n_estimators = np.logspace(start=round(math.log(train_dataset_size/20,2)), stop=round(math.log(train_dataset_size/4,2)),base=2,num=5).astype(int)
min_child_weight = np.logspace(start=round(math.log(train_dataset_size/20,2)), stop=round(math.log(train_dataset_size/4),2),base=2,num=5).astype(int)
learning_rate = np.logspace(start=-2,stop=0,num=6)
param_grid12 = {'n_estimators':n_estimators,'min_child_weight':min_child_weight,
               'learning_rate':learning_rate}

XGBModel3 = xgb.XGBClassifier(use_label_encoder=False, objective='binary:logistic', eval_metric='auc',
                             sampling_method='gradient_based',max_depth=4)

clf55 = GridSearchCV(estimator=XGBModel3,param_grid=param_grid12,cv=cv,scoring=scoring,return_train_score=True,verbose=3)
clf55.fit(df_cleaned2_train, target)
clf55.score(df_cleaned2_train, target)
clf55.best_estimator_

dump(clf55,  '../models/XGB_10cv_GRIDsearch_vardrop1.joblib')

from sklearn.ensemble import RandomForestClassifier

RFModel = RandomForestClassifier(random_state=42)
n_estimators = np.logspace(start=1, stop=round(math.log(train_dataset_size/2,2)),base=2,num=5).astype(int)
max_depth = np.arange(start=2,stop=5)
min_samples_split = np.logspace(start=1, stop=round(math.log(train_dataset_size/2),2),base=2,num=5).astype(int)
param_grid13 = {'max_depth':max_depth,'n_estimators':n_estimators,'min_samples_split':min_samples_split}

clf56 = GridSearchCV(estimator=RFModel,param_grid=param_grid13,cv=cv,scoring=scoring,return_train_score=True,verbose=3)
clf56.fit(df_cleaned2_train, target)
clf56.best_estimator_
clf56.score(df_cleaned2_train, target)
clf56.cv_results_.keys()

dump(clf56,  '../models/randforest_10cv_gridsearch_vardrop1.joblib')

RFModel = RandomForestClassifier(random_state=42,max_depth=4)
n_estimators = np.linspace(start=80, stop=100,num=11).astype(int)
min_samples_split = np.linspace(start=10, stop=30,num=6).astype(int)
param_grid14 = {'n_estimators':n_estimators,'min_samples_split':min_samples_split}

clf57 = GridSearchCV(estimator=RFModel,param_grid=param_grid14,cv=cv,scoring=scoring,return_train_score=True,verbose=3)
clf57.fit(df_cleaned2_train, target)
clf57.best_estimator_
clf57.score(df_cleaned2_train, target)
clf57.cv_results_.keys()

dump(clf57,  '../models/randforest_10cv_gridsearch_vardrop2.joblib')

RFModel = RandomForestClassifier(random_state=42,max_depth=4)
n_estimators = np.linspace(start=85, stop=95,num=11).astype(int)
min_samples_split = np.linspace(start=25, stop=45,num=11).astype(int)
param_grid15 = {'n_estimators':n_estimators,'min_samples_split':min_samples_split}

clf58 = GridSearchCV(estimator=RFModel,param_grid=param_grid15,cv=cv,scoring=scoring,return_train_score=True,verbose=3)
clf58.fit(df_cleaned2_train, target)
clf58.best_estimator_
clf58.score(df_cleaned2_train, target)
clf58.cv_results_.keys()

dump(clf58,  '../models/randforest_10cv_gridsearch_vardrop3.joblib')

RFModel = RandomForestClassifier(random_state=42,max_depth=4,n_estimators=88)
min_samples_split = np.linspace(start=25, stop=45,num=21).astype(int)
param_grid15 = {'min_samples_split':min_samples_split}

clf59 = GridSearchCV(estimator=RFModel,param_grid=param_grid15,cv=cv,scoring=scoring,return_train_score=True,verbose=3)
clf59.fit(df_cleaned2_train, target)
clf59.best_estimator_
clf59.score(df_cleaned2_train, target)
clf59.cv_results_.keys()

dump(clf59,  '../models/randforest_10cv_gridsearch_vardrop4.joblib')

RFModel = RandomForestClassifier(random_state=42)
max_depth = np.arange(start=2,stop=7)
n_estimators = np.linspace(start=75, stop=95,num=21).astype(int)
min_samples_split = np.linspace(start=25, stop=45,num=21).astype(int)
param_grid16 = {'max_depth':max_depth,'n_estimators':n_estimators,'min_samples_split':min_samples_split}

clf60 = GridSearchCV(estimator=RFModel,param_grid=param_grid16,cv=cv,scoring=scoring,return_train_score=True,verbose=3)
clf60.fit(df_cleaned2_train, target)
clf60.best_estimator_
clf60.score(df_cleaned2_train, target)
clf60.cv_results_.keys()

dump(clf60,  '../models/randforest_10cv_gridsearch_vardrop5.joblib')

roc_test = clf60.best_estimator_.predict_proba(df_cleaned2_test)[:,1]
code = BuildCSVforSubmission(IDlist_test,roc_test)

RFModel = RandomForestClassifier(random_state=42)
max_depth = np.arange(start=2,stop=7)
n_estimators = np.linspace(start=75, stop=95,num=6).astype(int)
min_samples_split = np.linspace(start=25, stop=45,num=6).astype(int)
param_grid16 = {'max_depth':max_depth,'n_estimators':n_estimators,'min_samples_split':min_samples_split}

clf61 = GridSearchCV(estimator=RFModel,param_grid=param_grid16,cv=cv,scoring=scoring,return_train_score=True,verbose=3)
clf61.fit(df_cleaned4_train, target)
clf61.best_estimator_
clf61.score(df_cleaned4_train, target)
clf61.cv_results_.keys()

dump(clf61,  '../models/randforest_10cv_gridsearch_vardrop6.joblib')

roc_test = clf60.best_estimator_.predict_proba(df_cleaned2_test)[:,1]
code = BuildCSVforSubmission(IDlist_test,roc_test)

RFModel = RandomForestClassifier(random_state=42)
max_depth = np.arange(start=2,stop=7)
n_estimators = np.linspace(start=75, stop=95,num=6).astype(int)
min_samples_split = np.linspace(start=25, stop=45,num=6).astype(int)
param_grid18 = {'max_depth':max_depth,'n_estimators':n_estimators,'min_samples_split':min_samples_split}

clf62 = GridSearchCV(estimator=RFModel,param_grid=param_grid18,cv=cv,scoring=scoring,return_train_score=True,verbose=3)
clf62.fit(df_cleaned6_train, target)
clf62.best_estimator_
clf62.score(df_cleaned6_train, target)
clf62.cv_results_.keys()

dump(clf62,  '../models/randforest_10cv_gridsearch_vardrop7.joblib')

RFModel = RandomForestClassifier(random_state=42)
max_depth = np.arange(start=2,stop=7)
n_estimators = np.linspace(start=95, stop=125,num=6).astype(int)
min_samples_split = np.linspace(start=25, stop=45,num=6).astype(int)
param_grid19 = {'max_depth':max_depth,'n_estimators':n_estimators,'min_samples_split':min_samples_split}

clf63 = GridSearchCV(estimator=RFModel,param_grid=param_grid19,cv=cv,scoring=scoring,return_train_score=True,verbose=3)
clf63.fit(df_cleaned6_train, target)
clf63.best_estimator_
clf63.score(df_cleaned6_train, target)
clf63.cv_results_.keys()

dump(clf63,  '../models/randforest_10cv_gridsearch_vardrop8.joblib')


## second version of dataset
RFModel = RandomForestClassifier(random_state=42)
n_estimators = np.logspace(start=1, stop=round(math.log(train_dataset_size/2,2)),base=2,num=5).astype(int)
max_depth = np.arange(start=2,stop=5)
min_samples_split = np.logspace(start=1, stop=round(math.log(train_dataset_size/2),2),base=2,num=5).astype(int)
param_grid21 = {'max_depth':max_depth,'n_estimators':n_estimators,'min_samples_split':min_samples_split}

clf71 = GridSearchCV(estimator=RFModel,param_grid=param_grid21,cv=cv,scoring=scoring,return_train_score=True,verbose=3)
clf71.fit(df_cleaned2_train, target)
clf71.best_estimator_
clf71.score(df_cleaned2_train, target)
clf71.cv_results_.keys()

dump(clf71,  '../models/randforest_10cv_gridsearch_vardrop11.joblib')

clf72 = GridSearchCV(estimator=RFModel,param_grid=param_grid21,cv=cv,scoring=scoring,return_train_score=True,verbose=3)
clf72.fit(df_cleaned4_train, target)
clf72.best_estimator_
clf72.score(df_cleaned4_train, target)
clf72.cv_results_.keys()

dump(clf72,  '../models/randforest_10cv_gridsearch_vardrop12.joblib')